Clasificación de Imágenes con métodos clásicos y redes neuronales

In [ ]:
#%pip install scikit-image -q
#%pip install skimage -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import resize
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore")


In [ ]:
def imagen(address):
    scale = 8
    img_width = int(1920/scale)
    img_height = int(1080/scale)
    rgb = io.imread(address)
    rgb_resized = resize(rgb, (img_height, img_width), anti_aliasing=True)   

    nbins = 16

    rh = np.histogram(rgb_resized[:,:,0].flatten(), nbins, density = True)
    gh = np.histogram(rgb_resized[:,:,1].flatten(), nbins, density = True)
    bh = np.histogram(rgb_resized[:,:,2].flatten(), nbins, density = True)
    hist_descriptor = np.concatenate((rh[0], gh[0], bh[0]))

    gray_resized = img_as_ubyte(rgb2gray(rgb_resized))
    glcm = greycomatrix(gray_resized, distances=[5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])
        
    texture_desc = [greycoprops(glcm, 'dissimilarity')[0, 0], greycoprops(glcm, 'homogeneity')[0, 0], 
    greycoprops(glcm, 'energy')[0, 0], greycoprops(glcm, 'correlation')[0, 0]]
    return np.concatenate((hist_descriptor, np.array(texture_desc)))

In [ ]:
list_agua = (glob.glob("C:/Users/arcto/Desktop/Images ML/Agua/*.jpg"))
list_bosque = (glob.glob("C:/Users/arcto/Desktop/Images ML/Bosque/*.jpg"))
list_ciudad = (glob.glob("C:/Users/arcto/Desktop/Images ML/Ciudad/*.jpg"))
list_cultivo = (glob.glob("C:/Users/arcto/Desktop/Images ML/Cultivo/*.jpg"))
list_desierto = (glob.glob("C:/Users/arcto/Desktop/Images ML/Desierto/*.jpg"))
list_montana = (glob.glob("C:/Users/arcto/Desktop/Images ML/Montana/*.jpg"))
list_total = [list_agua, list_bosque, list_ciudad, list_cultivo, list_desierto, list_montana]

In [ ]:
#   Load image
x  = []
y = []
for i in range(0, len(list_total)):
   for j in range(0, len(list_total[i])):
      x.append(imagen(list_total[i][j]))
      y.append(i)

In [ ]:
x[0]

array([8.60578341e+00, 1.53725412e+01, 7.16791781e+00, 3.37718121e+00,
       1.23841246e+00, 4.86800895e-01, 1.81423482e-01, 3.60593255e-02,
       4.50741569e-03, 1.57759549e-02, 6.76112354e-03, 2.25370785e-03,
       5.63426962e-03, 6.76112354e-03, 0.00000000e+00, 2.25370785e-03,
       2.53435826e-02, 1.58397391e-02, 2.85115304e-02, 3.61146052e-01,
       1.12462148e+00, 2.17796413e+00, 5.49955742e+00, 2.51550897e+01,
       1.67473562e+01, 1.18798043e-01, 9.50384347e-03, 1.58397391e-02,
       1.74237130e-02, 1.10878174e-02, 9.50384347e-03, 3.16794782e-03,
       3.26139089e-02, 1.44950706e-02, 3.08020250e-02, 1.63069544e-01,
       6.99387157e-01, 1.93328004e+00, 3.15811351e+00, 1.92639488e+01,
       3.13564615e+01, 1.91516120e+00, 5.61683986e-02, 2.17426059e-02,
       2.89901412e-02, 1.08713030e-02, 1.44950706e-02, 5.43565148e-03,
       1.75089046e+00, 4.85695407e-01, 1.49005147e-01, 7.38298836e-01])

In [ ]:
y[0]

0

Clasificador SVM lineal

In [ ]:
clf = SVC(kernel='linear')
parameter_space = {
    'C': np.linspace(0.1, 2, 20)
}
clf = GridSearchCV(clf, parameter_space, n_jobs=1, cv=5, scoring=['accuracy', 'recall_micro'], refit='accuracy')
results = clf.fit(x, y)
param_c = results.best_params_['C']

print('Best parameters')
print(f'C: {param_c}')

Best parameters
C: 0.2


In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
splits = 5
skf = StratifiedKFold(n_splits = splits)
acc = 0 
recall = np.zeros(6)

for train_index, test_index in skf.split(x, y):
  X_train, X_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]

  clf = SVC(kernel='linear', C=param_c)
  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)
  acc += accuracy_score(y_test, y_pred)
  recall += recall_score(y_test, y_pred, average = None)

print(f'Accuracy of the model: {acc/splits}')
print(f'Recall for each class: ')
print(recall/splits)

Accuracy of the model: 0.7440188683880795
Recall for each class: 
[0.7761194  0.79402985 0.8571993  0.53934871 0.84432384 0.65460358]


Modelo no lineal de SVM (Radial)

In [ ]:
clf = SVC(kernel='rbf')
parameter_space = {
    'C': np.linspace(0.1, 2, 10),
    'gamma': ['scale', 'auto']
}
clf = GridSearchCV(clf, parameter_space, n_jobs=1, cv=5, scoring=['accuracy', 'recall_micro'], refit='accuracy')
results = clf.fit(x, y)
param_c = results.best_params_['C']
param_gamma = results.best_params_['gamma']

print('Best parameters')
print(f'C: {param_c}')
print(f'gamma: {param_gamma}')

Best parameters
C: 2.0
gamma: scale


In [ ]:
splits = 5
skf = StratifiedKFold(n_splits = splits)
acc = 0 
recall = np.zeros(6)
for train_index, test_index in skf.split(x, y):
  X_train, X_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]

  clf = SVC(kernel='rbf', C=param_c, gamma=param_gamma)
  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)
  acc += accuracy_score(y_test, y_pred)
  recall += recall_score(y_test, y_pred, average = None)

print(f'Accuracy of the modelo: {acc/splits}')
print(f'Recall for each class: ')
print(recall/splits)

Accuracy of the modelo: 0.7504729381126699
Recall for each class: 
[0.7641791  0.7761194  0.83309921 0.50651289 0.89231117 0.73081841]


MLP

In [ ]:
clf = MLPClassifier(activation='relu', solver='adam')
parameter_space = {
    'hidden_layer_sizes': [(20, 20), (20, 20, 20), (20, 20, 20, 20)],
    'alpha': np.linspace(0.001, 2, 3),
}
clf = GridSearchCV(clf, parameter_space, n_jobs=1, cv=5, scoring=['accuracy', 'recall_micro'], refit='accuracy')
results = clf.fit(x, y)
param_layers = results.best_params_['hidden_layer_sizes']
param_alpha = results.best_params_['alpha']

print('Best parameters')
print(f'Layers: {param_layers}')
print(f'Alpha: {param_alpha}')

Best parameters
Layers: (20, 20, 20, 20)
Alpha: 1.0005


In [ ]:
splits = 5
skf = StratifiedKFold(n_splits = splits)
acc = 0 
recall = np.zeros(6)
for train_index, test_index in skf.split(x, y):
  X_train, X_test = x[train_index], x[test_index]
  y_train, y_test = y[train_index], y[test_index]

  clf = MLPClassifier(hidden_layer_sizes = param_layers, 
                      activation = 'relu', 
                      solver = 'adam', 
                      alpha = param_alpha)
  clf.fit(X_train, y_train)

  y_pred = clf.predict(X_test)
  acc += accuracy_score(y_test, y_pred)
  recall += recall_score(y_test, y_pred, average = None)

print(f'Accuracy of the modelo: {acc/splits}')
print(f'Recall for each class: ')
print(recall/splits)

Accuracy of the modelo: 0.7524433088470137
Recall for each class: 
[0.75820896 0.77910448 0.84201054 0.52397105 0.89828132 0.7127451 ]
